### Imports & Definitions

In [1]:
import sys
sys.path.append("../../utils")
from definitions import *
from parsing_helpers import get_num_diplomas

In [2]:
years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016]

### Convert .doc, .docx & .pdf into .txt

In [54]:
def convert_works_to_txt(year):
    base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
    base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

    processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
    processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)

    def convert_doc_to_txt(filename):
        output_file = processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt")
        if not output_file.is_file():
            subprocess.run(["textutil", "-convert", "txt", "-output", output_file.as_posix(), filename.as_posix()])

    def convert_pdf_to_txt(filename):
        output_file = processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt")
        if not output_file.is_file():
            subprocess.run(["pdftotext", filename.as_posix(), output_file.as_posix()])

    for ext in [".doc", ".docx", ".DOCX"]:
        for file in tqdm(base_diplomas_dir_path.glob(f"*{ext}"), desc="Docs..."):
            convert_doc_to_txt(file)

    
    for file in tqdm(base_diplomas_dir_path.glob(f"*.pdf"), desc="Pdfs..."):
        convert_pdf_to_txt(file)

In [55]:
for year in tqdm(years, desc="Years..."):
    convert_works_to_txt(year)

Docs...: 29it [00:00, 5552.58it/s]:00<?, ?it/s]
Error reading /Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/2023/work/44850_diploma.docx.  The file “44850_diploma.docx” couldn’t be opened.
Error reading /Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/2023/work/39877_diploma.docx.  The file isn’t in the correct format.
Docs...: 722it [00:00, 2865.11it/s]
Docs...: 1it [00:00, 329.40it/s]
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Syntax Error: Document stream is empty
Pdfs...: 881it [00:00, 3019.44it/s]
Docs...: 25it [00:00, 3663.53it/s]:00<00:03,  1.79it/s]
Docs...: 708it [00:00, 51967.30it/s]
Docs...: 1it [00:

### Making df

In [57]:
base_diplomas_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/")
new_rows = []
for year in tqdm(years, desc="Years..."):
    ids_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/ids.json")
    skipped_ids_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/ids.json")
    with open(ids_path, "r") as f:
        ids = json.load(f)
    with open(skipped_ids_path, "r") as f:
        skipped_ids = set(json.load(f))
    new_rows_year = []
    for id in ids:
        diploma_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/work_txt/{id}_diploma.txt")
        abstract_path = base_diplomas_spbu_dir_path.joinpath(f"{year}/work/{id}_abstract.txt")
        if not abstract_path.is_file() or not diploma_path.is_file():
            assert not diploma_path.is_file(), f"Strange for id = {id}"
            assert id in skipped_ids
            continue
        with open(diploma_path, "r") as f:
            diploma = f.read()
        with open(abstract_path, "r") as f:
            abstract = f.read()
        new_rows_year.append({
            "id": id,
            "year": year,
            "diploma": diploma,
            "abstract": abstract,
        })
    df = pd.DataFrame(new_rows_year)
    df.to_csv(base_diplomas_spbu_dir_path.joinpath(f"{year}/dataset.csv"), index=False)
    new_rows.extend(new_rows_year)
df = pd.DataFrame(new_rows)
df.to_csv(base_diplomas_spbu_dir_path.joinpath(f"dataset.csv"), index=False)
df.head()

Years...: 100%|██████████| 8/8 [00:58<00:00,  7.29s/it]


,id,year,diploma,abstract
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...


### Filter df: empty diplomas

In [34]:
df = pd.DataFrame([{"a": 1}, {"a": np.float64('nan')}, {"a": 3}])
df[df["a"].notnull()]

,a
0,1.0
2,3.0


In [35]:
DATASET_PATH = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/dataset.csv")
df = pd.read_csv(DATASET_PATH)
df = df[df["diploma"].notnull()]
df.to_csv(DATASET_PATH, index=False)


In [36]:
len(df)

13933

### Extend df with study_field & degree

In [3]:
df = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/dataset.csv"))
df.head()

,id,year,diploma,abstract
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...


In [41]:
new_rows = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Rows..."):
    id = row["id"]
    year = row["year"]
    with open(ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/view/{id}.html"), "r") as f:
        soup = BeautifulSoup(f.read())
    dict_soup = convert(soup)
    study_field = dict_soup["html"][0]["body"][0]["main"][0]["div"][1]["ol"][0]["li"][2]["#text"]
    degree = dict_soup["html"][0]["body"][0]["main"][0]["div"][1]["ol"][0]["li"][3]["#text"]
    new_row = copy.deepcopy(row)
    new_row["study_field"] = study_field
    new_row["degree"] = degree
    new_rows.append(new_row)

Rows...:   0%|          | 0/13933 [00:00<?, ?it/s]

In [42]:
new_df = pd.DataFrame(new_rows)
new_df.to_csv(DATASET_PATH, index=False)
new_df.head()

,id,year,diploma,abstract,study_field,degree
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...,MANAGEMENT,MASTER'S STUDIES
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES


In [43]:
new_df.__len__()

13933

### Add info about original extension

In [60]:
new_df.head()

,id,year,diploma,abstract,study_field,degree
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...,MANAGEMENT,MASTER'S STUDIES
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES


In [74]:
super_new_rows = []
for _, row in tqdm(new_df.iterrows(), total=len(new_df), desc="Rows..."):
    year, id = row["year"], row["id"]
    dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/work/")
    ext = None
    for path in dir_path.glob(f"{id}*"):
        if path.suffix != ".txt":
            ext = path.suffix
            break
    super_new_row = copy.deepcopy(row)
    super_new_row["original_diploma_extension"] = ext
    super_new_rows.append(super_new_row)

Rows...:   0%|          | 0/13933 [00:00<?, ?it/s]

In [75]:
super_new_df = pd.DataFrame(super_new_rows)
super_new_df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...,MANAGEMENT,MASTER'S STUDIES,.docx
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx


In [76]:
super_new_df.to_csv(DATASET_PATH, index=False)

### View if there is abstract in diploma text

In [90]:
super_new_df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...,MANAGEMENT,MASTER'S STUDIES,.docx
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx


In [94]:
def normalize(s):
    return " ".join(s.split())
normalize("The   fox jumped   over    the log.")

'The fox jumped over the log.'

In [96]:
num_inside = 0
num_inside_normalized = 0
not_inside_mask = []
for _, row in tqdm(super_new_df.iterrows(), total=len(super_new_df), desc="Rows..."):
    abstract, diploma = row["abstract"], row["diploma"]
    abstract_normalized = normalize(abstract)
    diploma_normalized = normalize(diploma)
    if abstract in diploma:
        num_inside += 1
        not_inside_mask.append(0)
    elif abstract_normalized in diploma_normalized:
        num_inside_normalized += 1
        not_inside_mask.append(0)
    else:
        not_inside_mask.append(1)
print(num_inside, num_inside_normalized, len(super_new_df))

Rows...:   0%|          | 0/13933 [00:00<?, ?it/s]

426 562 13933


In [99]:
not_inside_mask = np.array(not_inside_mask)
not_inside_mask

array([1, 1, 1, ..., 1, 0, 1])

In [104]:
super_puper_new_df = super_new_df.iloc[np.where(not_inside_mask)[0]]
super_puper_new_df.to_csv(DATASET_PATH, index=False)
super_puper_new_df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
0,40165,2023,St. Petersburg State University\nGraduate Scho...,Данная работа исследует потенциал использовани...,MANAGEMENT,MASTER'S STUDIES,.docx
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx


### View russian diplomas

In [6]:
df_2023 = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/russian_stupid_filter_dataset.csv"))

In [7]:
print(len(df_2023))
df_2023 = df_2023[df_2023["year"] == 2023]
len(df_2023)

10924


1387

### Calculating total capacity of diplomas

In [4]:
total = 0
for year in years:
    total += get_num_diplomas(year)
total

17137

### Filter df: remain russian (stupid)

In [111]:
(russian_stupid_filter_df["year"] == 2023).sum()

1271

In [105]:
pattern = "выпускная квалификационная работа"
lower_texts = list(map(lambda x: x.lower(), super_puper_new_df["diploma"]))
mask = []
for lower_text in lower_texts:
    mask.append(pattern in lower_text)

In [48]:
sum(mask), len(mask), sum(mask) / len(mask) * 100

(10924, 13933, 78.40378956434364)

In [49]:
np.where(mask)[0]

array([    1,     2,     3, ..., 13927, 13929, 13930])

In [106]:
russian_stupid_filter_df = super_puper_new_df.iloc[np.where(mask)[0]]

In [110]:
russian_stupid_filter_df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
1,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
2,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
3,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
4,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx
6,40166,2023,Федеральное государственное бюджетное образова...,На сегодняшний день наблюдается высокий спрос ...,MANAGEMENT,BACHELOR STUDIES,.docx


In [108]:
russian_stupid_filter_df.__len__()

10291

In [109]:
russian_stupid_filter_df.to_csv(ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/russian_dataset.csv"), index=False)

### [old] Convert .doc to .txt

In [4]:
base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

exts = defaultdict(int)
for file in base_diplomas_dir_path.glob("*"):
    exts[file.suffix] += 1
print(exts)

for ext in exts:
    for file in base_diplomas_dir_path.glob(f"*{ext}"):
        if ext != ".pages":
            break

processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)
def convert_doc_to_txt(filename):
    subprocess.run(["textutil", "-convert", "txt", "-output", processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix(), filename])

# for ext in [".doc", ".docx", ".DOCX"]:
#     for file in base_diplomas_dir_path.glob(f"*{ext}"):
#         convert_doc_to_txt(file)

defaultdict(<class 'int'>, {'.txt': 1654, '.pdf': 881, '.docx': 722, '.doc': 29, '.jpg': 4, '.spv': 1, '.jpeg': 7, '.pages': 1, '.mp4': 1, '.DOCX': 1, '.xlsx': 2, '.rtf': 1, '.odt': 1, '.docm': 1, '.JPG': 1, '.PDF': 1})


In [8]:
def convert_pdf_to_txt(filename):
    subprocess.run(["pdftotext", filename.as_posix(), processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix()])

for file in base_diplomas_dir_path.glob(f"*.pdf"):
    convert_pdf_to_txt(file)

Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Error: Document stream is empty
Syntax Warning: Invalid Font Weight
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warnin

### Filter diplomas: only russian

In [3]:
RAW_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/work/")
PROCESSED_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/stupid_filtered_work")
PROCESSED_DIPLOMAS_DIR_PATH.mkdir(exist_ok=True, parents=True)

In [3]:
num_ok, num_all = 0, 0
for file in RAW_DIPLOMAS_DIR_PATH.glob("*.txt"):
    with open(file, "r") as f:
        text = f.read()
    if "выпускная квалификационная работа" in text.lower() and "отзыв" not in text.lower():
        with open(PROCESSED_DIPLOMAS_DIR_PATH.joinpath(f"{file.stem}.txt"), "w") as f:
            f.write(text)
        print(text[:10])
        num_ok += 1
    num_all += 1
print(num_ok, num_all)

Санкт-Пете
          
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
ФГОУ ВО «С
Санкт-Пете
Санкт-Пете
Санкт-Пете
САНКТ-ПЕТЕ
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете

САНКТ-ПЕТ
Санкт-Пете
Санкт-Пете
23 143
